In [1]:
import numpy as np
import cv2

In [45]:
def erod(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    result = image.copy()
    if type_of_mask == 'square':
        result = erod_square(result, size)
    elif type_of_mask == 'cross':
        result = erod_cross(result, size)
    return result

In [178]:
def erod_square(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i + k, j] == 0 and image[i - k, j] == 0 and image[i, j + k] == 0 
                and image[i + k, j + k] == 0 and image[i - k, j + k] == 0 and image[i, j - k] == 0 and image[i + k, j - k] == 0
                and image[i - k, j - k] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result

In [179]:
def erod_cross(image, size):
    result = image.copy()
    status = True
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status and image[i + k, j] == 0 and image[i - k, j] == 0
                         and image[i, j + k] == 0 and image[i, j - k] == 0)
                if not status:
                    break
            if not status:
                result[i, j] = 255
            status = True
    return result

In [76]:
def dilat(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    if type_of_mask == 'square':
        result = dilat_square(result, size)
    elif type_of_mask == 'cross':
        result = dilat_cross(result, size)
    return result

In [193]:
def dilat_square(image, size):
    result = image.copy()
    status = False
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status or image[i + k, j] == 0 or image[i - k, j] == 0 or image[i, j + k] == 0
                         or image[i + k, j + k] == 0 or image[i - k, j + k] == 0 or image[i,j - k] == 0
                         or image[i + k, j - k] == 0 or image[i - k, j - k] == 0)
                if status:
                    break
            if status:
                result[i, j] = 0
            status = False
    return result

In [194]:
def dilat_cross(image, size):
    result = image.copy()
    status = False
    for i in range(result.shape[0] - size):
        for j in range(result.shape[1] - size):
            for k in range(1, int((size + 1) / 2)):
                status = (status or image[i + 1, j] == 0 or image[i - 1, j] == 0 or image[i, j + k] == 0
                         or image[i, j - k] == 0)
                if status:
                    break
            if status:
                result[i, j] = 0
            status = False
    return result

In [79]:
def opening(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = erod(result, size, type_of_mask)
    result = dilat(result, size, type_of_mask)
    return result

In [82]:
def closing(image, size, type_of_mask = 'square'):
    assert type_of_mask == 'square' or type_of_mask == 'cross'
    
    result = image.copy()
    result = dilat(result, size, type_of_mask)
    result = erod(result, size, type_of_mask)
    return result

In [85]:
def filter(image):
    result = image.copy()
    for i in range(3, image.shape[0] - 3):
        for j in range(3, image.shape[1] - 3):
            if image[i, j] == 0:
                if image[i, j + 1] == 255 and image[i, j - 1] == 255 and image[i + 1, j] == 255 and image[i - 1, j] == 255:
                    image[i, j] = 255
            elif image[i, j] == 255:
                if image[i, j + 1] == 0 and image[i, j - 1] == 0 and image[i + 1, j] == 0 and image[i - 1, j] == 0:
                    image[i, j] = 0
    return result

In [200]:
def make_noise(image, prob):
    result = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            p = np.random.randint(0, 101)
            if p < prob :
                if image[i, j] == 0:
                    result[i, j] = 255
                #elif image[i, j] == 255:
                #   result[i, j] = 0
    return result

In [98]:
def circle(image, centre, radius):
    circle = [centre, radius]
    for i in range(circle[0][0] - circle[1], circle[0][0] + circle[1]):
        for j in range(circle[0][1] - circle[1], circle[0][1] + circle[1]):
            if (i - circle[0][0]) ** 2 + (j - circle[0][1]) ** 2 < circle[1] ** 2:
                image[i, j] = 0

In [94]:
def triangle(image, A, B, C):
    triangle = [A, B, C]
    x_a = triangle[0][1]
    y_a = triangle[0][0]
    x_b = triangle[1][1]
    y_b = triangle[1][0]
    x_c = triangle[2][1]
    y_c = triangle[2][0]

    left = triangle[0][0]
    right = triangle[0][0]
    above = triangle[0][1]
    below = triangle[0][1]
    for i in triangle:
        if left > i[0]:
            left = i[0]
        if right < i[0]:
            right = i[0]
        if above > i[1]:
            above = i[1]
        if below < i[1]:
            below = i[1]
    for i in range(above, below):
        for j in range(left, right):
            coord_1 = (x_a - i) * (y_b - y_a) - (x_b - x_a) * (y_a - j)
            coord_2 = (x_b - i) * (y_c - y_b) - (x_c - x_b) * (y_b - j)
            coord_3 = (x_c - i) * (y_a - y_c) - (x_a - x_c) * (y_c - j)
            if coord_1 >= 0 and coord_2 >= 0 and coord_3 >= 0:
                image[i, j] = 0

In [89]:
def square(image, A, B):
    square = [A, B]
    for i in range(square[0][0], square[1][0]):
        for j in range(square[0][1], square[1][1]):
            image[i, j] = 0

In [205]:
image = np.zeros((500, 500))
image.fill(255)

In [206]:
square(image, (10, 10), (50, 50))
triangle(image, (100, 100), (100, 200), (200, 100))
circle(image, (400, 400), 40)
circle(image, (200, 250), 1)

In [207]:
cv2.imwrite('/home/qwerty/image.png', image)

True

In [208]:
noised = []
for i in [1, 2, 5, 10, 20, 50]:
    noise = make_noise(image, i)
    noised.append(noise)
    cv2.imwrite('/home/qwerty/image_noise_' + str(i) + '.png', noise)

In [181]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'square')
    cv2.imwrite('/home/qwerty/erod_image_square_' + str(i) + '.png', erod_image)

In [183]:
for i in range(3, 6):
    erod_image = erod(image, i, type_of_mask = 'cross')
    cv2.imwrite('/home/qwerty/erod_image_cross_'+ str(i) + '.png', erod_image)

In [185]:
for i in range(3, 6):
    dilat_image = dilat(image, i, type_of_mask = 'square')
    cv2.imwrite('/home/qwerty/dilat_image_square_' + str(i) + '.png', dilat_image)

In [186]:
for i in range(3, 6):
    dilat_image= dilat(image, i, type_of_mask = 'cross')
    cv2.imwrite('/home/qwerty/dilat_image_cross_' + str(i) + '.png', dilat_image)

In [209]:
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'square')
        cv2.imwrite('/home/qwerty/open_image_square' + str(j) + '_' + str(i) + '.png', open_image)

In [210]:
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = closing(noised[j], i, type_of_mask = 'square')
        cv2.imwrite('/home/qwerty/close_image_square' + str(j) + '_' + str(i) + '.png', open_image)

In [211]:
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = opening(noised[j], i, type_of_mask = 'cross')
        cv2.imwrite('/home/qwerty/open_image_cross' + str(j) + '_' + str(i) + '.png', open_image)

In [212]:
for i in range(3, 6):
    for j in range(len(noised)):
        open_image = closing(noised[j], i, type_of_mask = 'cross')
        cv2.imwrite('/home/qwerty/close_image_cross' + str(j) + '_' + str(i) + '.png', open_image)

In [214]:
for i in range(len(noised)):
    filter_image = filter(noised[i])
    cv2.imwrite('/home/qwerty/filter_image_' + str(i) + '.png', filter_image)